Import saved reaction datas

In [1]:
from __future__ import print_function

import os
import re
import cPickle, gzip
import progressbar
from rdkit.Chem import AllChem

In [2]:
data_directory = 'data'
processed_data_filename = 'processed_data.pkl.gz'
processed_data_filepath = os.path.join(data_directory, processed_data_filename)
data_length = 1094235

bar = progressbar.ProgressBar(max_value=data_length)

rsmi_data = []

with gzip.open(processed_data_filepath, 'rb') as data_file:
    i = 1
    while 1:
        try:
            line, ref = cPickle.load(data_file)
            rsmi_data.append((line, ref))
        except EOFError:
            break
        bar.update(i)
        i += 1

 99% (1088828 of 1094235) |################ | Elapsed Time: 0:00:13 ETA: 0:00:00

In [15]:
import parser.Smipar as Smipar

bar = progressbar.ProgressBar(max_value=data_length)

_PAD = "_PAD"
_GO = "_GO"
_EOS = "_EOS"
_START_VOCAB = [_PAD, _GO, _EOS]

PAD_ID = 0
GO_ID = 1
EOS_ID = 2

vocab_reactants = {}
vocab_products = {}

for i, rsmi in enumerate(rsmi_data):
    
    rsmi = rsmi[0]
    
    reactant_list = []
    agent_list = []
    product_list = []

    split_rsmi = rsmi.split('>')
    reactants = split_rsmi[0].split('.')
    agents = split_rsmi[1].split('.')
    products = split_rsmi[2].split('.')

    for reactant in reactants:
        reactant_list += Smipar.parser_list(reactant)
        reactant_list += '.'
    for agent in agents:
        agent_list += Smipar.parser_list(agent)
        agent_list += '.'
    for product in products:
        product_list += Smipar.parser_list(product)
        product_list += '.'
      
    reactant_list.pop() # to pop last '.'
    agent_list.pop()
    product_list.pop()
    
    reactant_list += '>'
    reactant_list += agent_list
    
    for reactant_token in reactant_list:
        if reactant_token in vocab_reactants:
            vocab_reactants[reactant_token] += 1
        else:
            vocab_reactants[reactant_token] = 1
    
    for product_token in product_list:
        if product_token in vocab_products:
            vocab_products[product_token] += 1
        else:
            vocab_products[product_token] = 1
            
    bar.update(i)

 99% (1094226 of 1094235) |################ | Elapsed Time: 3:45:15 ETA: 0:00:00

In [18]:
reactants_token_list = _START_VOCAB \
        + sorted(vocab_reactants, key=vocab_reactants.get, reverse=True)

products_token_list = _START_VOCAB \
        + sorted(vocab_products, key=vocab_products.get, reverse=True)
    
with gzip.open('data/vocab/vocab_dict.pkl.gz', 'wb') as dict_file:
    cPickle.dump((vocab_reactants, vocab_products), dict_file, 2)
    
with gzip.open('data/vocab/vocab_list.pkl.gz', 'wb') as list_file:
    cPickle.dump((reactants_token_list, products_token_list), list_file, 2)

In [ ]:
with gzip.open('data/vocab/vocab_list.pkl.gz', 'rb') as list_file:
    reactants_token_list, products_token_list = cPickle.load(list_file)